### Load Packages

In [ ]:
%matplotlib inline

import os
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

data_path = os.path.join('..','Data')

from indirect import run_model

pd.options.display.float_format = '{:.3f}'.format

### Create mapping dictionary so we have names instead of codes

In [ ]:
sector_codes = ['A','B','C1','C2','C3','C4','C5','C6','C7','C8','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T']
full_sector_names = ['Agriculture, forestry and fishing','Quarrying',
'General Industry','Food industry','Oil refinery','Chemical & biobased industry','Basic metal-metal products industry',
'Means of transport ind. and maritime service','Electricity generation','Other Industry',
'Construction industry','Trade','Transport and storage','Catering industry',
'Information and communication','Financial services','Rental and trading of real estate',
'Specialist Business Services','Rental and other business services','Public administration and government services',
'Education','Health and welfare care','Culture, sport and recreation','Other services','Household services']

sector_dict = dict(zip(sector_codes,full_sector_names))

### Prepare reconstruction curves

In [ ]:
EPSILON = sys.float_info.epsilon  # smallest possible difference

def get_recon_curve(ini_curve,timesteps):
    if timesteps == 0:
        return [0,0]

    def interpolate(inp, fi):
        i = int(fi)
        f = fi - i
        return (inp[i] if f < EPSILON else
                inp[i] + f*(inp[i+1]-inp[i]))

    inp = ini_curve
    new_len = timesteps

    delta = (len(inp)-1) / float(new_len-1)
    outp = np.diff([1-int(interpolate(inp, i*delta))/100 for i in range(new_len)])
    #outp = [1-int(interpolate(inp, i*delta))/100 for i in range(new_len)]
    return list(outp)

### Load IO Data

In [ ]:
IO_TABLE = pd.read_csv(os.path.join(data_path,'Rijnmond_IO.csv'),index_col=[0],header=[0])
SECTORS = list(IO_TABLE[:25].index.get_level_values(0).unique())

### Load direct impacts

In [ ]:
output_path = os.path.join(data_path,'..','output')
damages_sector = pd.read_csv(os.path.join(output_path,'damages_sector.csv'),header=[0],index_col=[0])

### Translate direct impacts to relative impacts

In [ ]:
ValueA = IO_TABLE.iloc[25,:25]*1e6

In [ ]:
damages_sector = damages_sector.merge(ValueA,left_index=True,right_index=True)
damages_sector['disruption'] = damages_sector['damages'].div(damages_sector.Z_BrTW)
damages_sector = damages_sector.drop('Z_BrTW',axis=1)

### Set maximum recovery duration

In [ ]:
recon_time = 360
inventory = 3

### Run ARIO Model for different recovery paths

In [ ]:
%%time
diff_recons_tot = {}
diff_recons_ind = {}
diff_recons_path = {}
curve_types = ['linear','convex','concave']
concave = np.array([100,98,93,88,83,75,64,53,38,20,0])
linear = np.array([100,90,80,70,60,50,40,30,20,10,0])
convex = np.array([100,70,55,40,30,20,15,10,5,0,0])

for recon_type,curve_type in tqdm(zip([linear,convex,concave],curve_types),total=len(curve_types)): #
       
    # create reconstruction matrix
    all_sectors = [get_recon_curve(recon_type,int(x)) for x in list([recon_time]*25)]
    pad = max(len(max(all_sectors, key=len))+1,recon_time)
    recon_matrix = np.array([i + [0]*(pad-len(i)) for i in all_sectors])


    # perform calculations
    new_df = damages_sector.copy()
    new_df.loc[new_df.disruption > 1] = 0.99
    rel_impact = dict(zip(new_df.index,new_df.disruption))

    # total losses
    get_losses_tot = (run_model([rel_impact[x] if x in rel_impact.keys() else 0 for x in SECTORS],recon_matrix,inventory,pad))[0]
    diff_recons_tot[curve_type] = get_losses_tot
    
    # loss per industry
    get_losses_ind = (run_model([rel_impact[x] if x in rel_impact.keys() else 0 for x in SECTORS],recon_matrix,inventory,pad))[1]
    diff_recons_ind[curve_type] = get_losses_ind
    
    # total losses over time
    get_losses_path = (run_model([rel_impact[x] if x in rel_impact.keys() else 0 for x in SECTORS],recon_matrix,inventory,pad))[2]
    diff_recons_path[curve_type] = get_losses_path


### Print the total damage for each recovery path
Print the *diff_recons_tot* variable below to view the total losses per recovery curve

And you can do the same with the *diff_recons_ind* variable to view the losses per curve, per sector. In the example below we plot the linear curve, but you can change this to the convex or concave curve.

In [ ]:
loss_per_sector = pd.DataFrame.from_dict(dict(zip(SECTORS,diff_recons_ind['linear'])),
                                         orient='index',columns=['losses'])
loss_per_sector.index = loss_per_sector.index.map(sector_dict)

### And plot a figure with the recovery paths back to the pre-disaster situation

In [ ]:
plt.figure()

plt.plot(diff_recons_path['linear'])
plt.plot(diff_recons_path['convex'])
plt.plot(diff_recons_path['concave'])